#06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
# 2015-summary.json

In [4]:
df = spark.read.format("json")\
                .load("learning_spark_data/2015-summary.json")

In [6]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [8]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [19]:
df.count()

256

In [10]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [11]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [12]:
df.select('DEST_COUNTRY_NAME').show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [ ]:
df.select('DEST_COUNTRY_NAME').show(5)

In [21]:
# 도시국가명 중복제거
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.count()

132

In [22]:
# ROW CLASS를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [25]:
# 새로운 컬럼 추기하기
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME'))
df3.show(5)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [ ]:
df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME'))
df3.show(5)

In [27]:
df3.filter(df3.withinCountry == True).show()
# df3.where('withinCountry == true').show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [ ]:
# case when count가 10이하이면 under, 10이상이면 upper로 변환 -> category 컬럼 추가

In [33]:
df4 = df3.withColumn(
    'category', expr("CASE WHEN count <= 10 THEN 'under' ELSE 'upper' END"))

df4.show(5)


+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 5 rows



In [ ]:
# Dataframe의 select(), where(), filter() 트렌스포메이션
# show(), count() 액션

In [34]:
spark.stop()

In [ ]:
# 집계함수 dept,emp

In [ ]:
emp_df, dept_df

In [45]:
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [52]:
dept_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("learning_spark_data/dept.csv")

dept_df.show(5)

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [51]:
emp_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("learning_spark_data/emp.csv")

emp_df.show(5)

+-----+------+--------+----+----------+----+----+------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+------+--------+----+----------+----+----+------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES| MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN|SALESMAN|7698|1981-09-28|1250|1400|    30|
+-----+------+--------+----+----------+----+----+------+
only showing top 5 rows



In [ ]:
# filter랑 동일함
emp_df.select('*')

In [53]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [56]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [59]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job',0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [ ]:
# first, last, min, max, sum, avg ->expr:sql 문장 x , function으로 처리

In [71]:
from pyspark.sql.functions import first, last, min, max, sum, avg

emp_df.select(first('sal'),
              last('sal'),
               min('sal'),
               max('sal'),
               sum('sal'),
              avg('sal')).show()

+----------+---------+--------+--------+--------+------------------+
|first(sal)|last(sal)|min(sal)|max(sal)|sum(sal)|          avg(sal)|
+----------+---------+--------+--------+--------+------------------+
|       800|     3200|     800|    5000|   32225|2148.3333333333335|
+----------+---------+--------+--------+--------+------------------+



In [70]:
# expr
from pyspark.sql.functions import expr

emp_df.select(
    expr("first(sal)"),
    expr("last(sal)"),
    expr("min(sal)"),
    expr("max(sal)"),
    expr("sum(sal)"),
    expr("avg(sal)"),
).show()


+----------+---------+--------+--------+--------+------------------+
|first(sal)|last(sal)|min(sal)|max(sal)|sum(sal)|          avg(sal)|
+----------+---------+--------+--------+--------+------------------+
|       800|     3200|     800|    5000|   32225|2148.3333333333335|
+----------+---------+--------+--------+--------+------------------+



In [75]:
emp_df.selectExpr('sum(DISTINCT sal)').show()


+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [ ]:
# total_salary, total_transaction, avg_salary, mean_sala

In [78]:
emp_df.selectExpr(
    'sum(sal) as total_salary',
    'sum(comm) as total_transaction',
    'avg(sal) as avg_salary',
    'avg(sal) as mean_sala'
).show()

+------------+-----------------+------------------+------------------+
|total_salary|total_transaction|        avg_salary|         mean_sala|
+------------+-----------------+------------------+------------------+
|       32225|             2200|2148.3333333333335|2148.3333333333335|
+------------+-----------------+------------------+------------------+



In [79]:
emp_df.selectExpr(
    'sum(sal) as total_salary',
    'sum(comm) as total_transaction',
    'ROUND(avg(sal), 2) as avg_salary',
    'ROUND(avg(sal), 2) as mean_sala'
).show()


+------------+-----------------+----------+---------+
|total_salary|total_transaction|avg_salary|mean_sala|
+------------+-----------------+----------+---------+
|       32225|             2200|   2148.33|  2148.33|
+------------+-----------------+----------+---------+



In [ ]:
# 그룹화

In [80]:
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [82]:
# select job,
#     count(job),
#     sum(sal)
# group by job
from pyspark.sql.functions import count, sum

group_df = emp_df.groupby('job').agg(
    count('job').alias('qty'),
    sum('sal').alias('total_salary')
)

group_df.show()


+---------+---+------------+
|      job|qty|total_salary|
+---------+---+------------+
|  ANALYST|  2|        6000|
| SALESMAN|  4|        5600|
|    CLERK|  5|        7350|
|  MANAGER|  3|        8275|
|PRESIDENT|  1|        5000|
+---------+---+------------+



In [84]:
# sal의 평균 SAL,AVG, 표준편차 SAL_STDEV를 job별로 계산해서 출력, 소수점 2자리

from pyspark.sql.functions import avg, stddev, round

group_job_df = emp_df.groupBy('job').agg(
    round(avg('sal'), 2).alias('AVG_SAL'),
    round(stddev('sal'), 2).alias('SAL_STDEV')
)

group_job_df.show()


+---------+-------+---------+
|      job|AVG_SAL|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [89]:
from pyspark.sql.functions import col

sal_top10_df = emp_df.select("job", "sal") \
                     .orderBy(col("sal").desc()) \
                     .limit(10)

sal_top10_df.show()

+---------+----+
|      job| sal|
+---------+----+
|PRESIDENT|5000|
|    CLERK|3200|
|  ANALYST|3000|
|  ANALYST|3000|
|  MANAGER|2975|
|  MANAGER|2850|
|  MANAGER|2450|
| SALESMAN|1600|
| SALESMAN|1500|
|    CLERK|1300|
+---------+----+



In [96]:
# 윈도우 함수
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank


Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [95]:
emp_df.withColumn('slary_rank', salAllRank).show()

+-----+------+---------+----+----------+----+----+------+----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|slary_rank|
+-----+------+---------+----+----------+----+----+------+----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|         1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|         2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|         3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|         3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|         5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|         6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|         7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|         8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|         9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|        11|
| 7654|MARTIN| SALESMAN|7698|1981-

In [98]:
#직무별로 rank작성
#window, partitionBy
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc, col

# 부서별로 나누고, 각 부서 내에서 급여 내림차순 정렬하여 순위 계산
windowSpec = Window.partitionBy("deptno").orderBy(desc("sal"))

ranked_df = emp_df.withColumn("dept_sal_rank", rank().over(windowSpec))

ranked_df.select("empno", "deptno", "sal", "dept_sal_rank").show()


+-----+------+----+-------------+
|empno|deptno| sal|dept_sal_rank|
+-----+------+----+-------------+
| 7839|    10|5000|            1|
| 7782|    10|2450|            2|
| 7934|    10|1300|            3|
| 7788|    20|3000|            1|
| 7902|    20|3000|            1|
| 7566|    20|2975|            3|
| 7876|    20|1100|            4|
| 7369|    20| 800|            5|
| 7698|    30|2850|            1|
| 7499|    30|1600|            2|
| 7844|    30|1500|            3|
| 7521|    30|1250|            4|
| 7654|    30|1250|            4|
| 7900|    30| 950|            6|
| 9292|    70|3200|            1|
+-----+------+----+-------------+



In [99]:
# 부서별 순위
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

# 부서별로 파티션을 나누고, sal(급여) 내림차순으로 정렬
windowSpec = Window.partitionBy("deptno").orderBy(desc("sal"))

# 부서별 순위를 rank()로 계산
ranked_df = emp_df.withColumn("dept_salary_rank", rank().over(windowSpec))

# 결과 출력
ranked_df.select("empno", "ename", "deptno", "sal", "dept_salary_rank").show()


+-----+------+------+----+----------------+
|empno| ename|deptno| sal|dept_salary_rank|
+-----+------+------+----+----------------+
| 7839|  KING|    10|5000|               1|
| 7782| CLARK|    10|2450|               2|
| 7934|MILLER|    10|1300|               3|
| 7788| SCOTT|    20|3000|               1|
| 7902|  FORD|    20|3000|               1|
| 7566| JONES|    20|2975|               3|
| 7876| ADAMS|    20|1100|               4|
| 7369| SMITH|    20| 800|               5|
| 7698| BLAKE|    30|2850|               1|
| 7499| ALLEN|    30|1600|               2|
| 7844|TURNER|    30|1500|               3|
| 7521|  WARD|    30|1250|               4|
| 7654|MARTIN|    30|1250|               4|
| 7900| JAMES|    30| 950|               6|
| 9292|  JACK|    70|3200|               1|
+-----+------+------+----+----------------+



In [100]:
# 누적 급여 sum('sal').over()
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, desc

# 전체 데이터를 sal 기준으로 내림차순 정렬한 윈도우 스펙 정의
windowSpec = Window.orderBy(desc('sal'))

# 누적 급여 계산 (내림차순 누적합)
emp_df.withColumn("cum_salary", sum('sal').over(windowSpec)) \
      .select("empno", "ename", "sal", "cum_salary") \
      .show()


+-----+------+----+----------+
|empno| ename| sal|cum_salary|
+-----+------+----+----------+
| 7839|  KING|5000|      5000|
| 9292|  JACK|3200|      8200|
| 7788| SCOTT|3000|     14200|
| 7902|  FORD|3000|     14200|
| 7566| JONES|2975|     17175|
| 7698| BLAKE|2850|     20025|
| 7782| CLARK|2450|     22475|
| 7499| ALLEN|1600|     24075|
| 7844|TURNER|1500|     25575|
| 7934|MILLER|1300|     26875|
| 7521|  WARD|1250|     29375|
| 7654|MARTIN|1250|     29375|
| 7876| ADAMS|1100|     30475|
| 7900| JAMES| 950|     31425|
| 7369| SMITH| 800|     32225|
+-----+------+----+----------+



In [101]:
# 부서별 누적급여
windowSpec = Window.partitionBy("deptno").orderBy(desc("sal"))

emp_df.withColumn("dept_cum_salary", sum('sal').over(windowSpec)) \
      .select("empno", "ename", "deptno", "sal", "dept_cum_salary") \
      .show()


+-----+------+------+----+---------------+
|empno| ename|deptno| sal|dept_cum_salary|
+-----+------+------+----+---------------+
| 7839|  KING|    10|5000|           5000|
| 7782| CLARK|    10|2450|           7450|
| 7934|MILLER|    10|1300|           8750|
| 7788| SCOTT|    20|3000|           6000|
| 7902|  FORD|    20|3000|           6000|
| 7566| JONES|    20|2975|           8975|
| 7876| ADAMS|    20|1100|          10075|
| 7369| SMITH|    20| 800|          10875|
| 7698| BLAKE|    30|2850|           2850|
| 7499| ALLEN|    30|1600|           4450|
| 7844|TURNER|    30|1500|           5950|
| 7521|  WARD|    30|1250|           8450|
| 7654|MARTIN|    30|1250|           8450|
| 7900| JAMES|    30| 950|           9400|
| 9292|  JACK|    70|3200|           3200|
+-----+------+------+----+---------------+



In [102]:
# 부서별 평균급여와 직원 개별 급여 비교
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, when, col

# 부서별 평균 급여 윈도우 정의
windowSpec = Window.partitionBy('deptno')

# 평균 급여 컬럼 추가 + 비교 결과 추가
emp_df.withColumn('avg_dept_sal', avg('sal').over(windowSpec)) \
      .withColumn('compare_result',
                  when(col('sal') > col('avg_dept_sal'), 'above_avg')
                  .when(col('sal') < col('avg_dept_sal'), 'below_avg')
                  .otherwise('equal')) \
      .select('empno', 'ename', 'deptno', 'sal', 'avg_dept_sal', 'compare_result') \
      .show()


+-----+------+------+----+------------------+--------------+
|empno| ename|deptno| sal|      avg_dept_sal|compare_result|
+-----+------+------+----+------------------+--------------+
| 7782| CLARK|    10|2450|2916.6666666666665|     below_avg|
| 7839|  KING|    10|5000|2916.6666666666665|     above_avg|
| 7934|MILLER|    10|1300|2916.6666666666665|     below_avg|
| 7369| SMITH|    20| 800|            2175.0|     below_avg|
| 7566| JONES|    20|2975|            2175.0|     above_avg|
| 7788| SCOTT|    20|3000|            2175.0|     above_avg|
| 7876| ADAMS|    20|1100|            2175.0|     below_avg|
| 7902|  FORD|    20|3000|            2175.0|     above_avg|
| 7499| ALLEN|    30|1600|1566.6666666666667|     above_avg|
| 7521|  WARD|    30|1250|1566.6666666666667|     below_avg|
| 7654|MARTIN|    30|1250|1566.6666666666667|     below_avg|
| 7698| BLAKE|    30|2850|1566.6666666666667|     above_avg|
| 7844|TURNER|    30|1500|1566.6666666666667|     below_avg|
| 7900| JAMES|    30| 95

In [ ]:
# sql 조인

In [103]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [ ]:
join_df = emp_df.join(dept_df, on='deptno', how='inner')
join_df.select('ename','','')

In [106]:
spark.stop()